## Stage 4. 데이터 전처리 (2)

### 1. 데이터 분석 전 준비

In [8]:
import pandas as pd
pd.options.mode.chained_assignment = None

all_data = pd.read_csv('../data/all_data.csv')

/tmp/ipykernel_578842/396092589.py:4: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  all_data = pd.read_csv('../data/all_data.csv')


In [9]:
all_data.columns

Index(['시군구', '번지', '아파트명', '전용면적', '계약년월', '계약일', '층', '건축년도', '도로명',
       '해제사유발생일', '거래유형', '중개사소재지', 'k-단지분류', 'k-세대타입(분양형태)', 'k-복도유형',
       'k-난방방식', 'k-전체동수', 'k-전체세대수', 'k-건설사', 'k-시행사', 'k-연면적', 'k-주거전용면적',
       'k-전용면적별세대현황60이하', 'k-전용면적별세대현황6085이하', 'k-85135이하', 'k-135초과',
       '세대전기계약방법', '건축면적', '주차대수', '기타의무임대1234', '단지신청일', 'target', '구', '동',
       '전체주소명', '동아파트명', '유동인구', '아파트 지하철역 거리', '1차역세권', '2차역세권',
       '아파트 버스정류장 거리', '좌표X', '좌표Y', '아파트 한강대교 거리', '전세가율', '전체계약일자', '금리',
       '실질gdp', '명목gdp', '학군', '재개발개수', '학급수', '여당득표율', 'train_test', '계약날짜',
       'apartment_id', 'bucket_area', 'recent_price', 'transaction_cnt',
       '1년전_금리'],
      dtype='object')

### 2. 아파트 이름 전처리 (1)

[K-means Clustering]
- 아파트 이름만으로 아파트 실거래가를 예측하기는 어려움
- 아파트의 과거 가격 순서대로 정렬 후, 비슷한 아파트 끼리 묶어 그룹을 만들어 줌(K-Means clsutering)
- 아파트의 target 값이 높은 순서대로 숫자를 부여하여 학습에 적용시킴

In [6]:
from sklearn.cluster import KMeans
import numpy as np

# 아파트 별로 가격 평균값 구하기
train = all_data[all_data['train_test'] == 'train']
train['target'] = train['target'].astype('int')
data = train[['아파트명', 'target']]

data = data.groupby('아파트명').mean()
arr = data['target'].to_numpy().reshape(-1, 1)

# 가격을 기준으로 아파트 군집화
k = 5  
kmeans = KMeans(n_clusters=k, n_init=10)
kmeans.fit(arr)

# 가격을 기준으로 군집의 순서를 정렬하기 위해 인덱스를 추출
sort_order = np.argsort(kmeans.cluster_centers_.flatten())

# 군집화 결과를 가격 순서대로 재할당
labels = np.zeros_like(kmeans.labels_)
for i, cluster in enumerate(sort_order):
    labels[kmeans.labels_ == cluster] = i

# 군집화 결과와 가격을 데이터에 추가
data['cluster'] = labels
data = data.reset_index()
data = data[['아파트명', 'cluster']]
data

,아파트명,cluster
0,153-25,0
1,201호 미공시,0
2,2차한양,1
3,3RU-City,0
4,3차신성미소지움,0
...,...,...
6533,힐탑트레져1동,3
6534,힐튼,0
6535,힐튼빌리지1차,1
6536,힐튼빌리지2차,1


### 3. 아파트 이름 전처리 (2)

In [7]:
# 아파트명을 기준으로 Left Joing 실행
all_data = pd.merge(all_data, data, how='left', left_on='아파트명', right_on='아파트명')

cluster_mode = all_data.loc[all_data['train_test'] == 'train', 'cluster'].mode()
all_data['cluster'] = all_data['cluster'].fillna(cluster_mode)

all_data['cluster'].value_counts()

0.0    688230
1.0    336426
2.0     91373
3.0      9784
4.0        73
Name: cluster, dtype: int64

### 4. 아파트 면적 전처리

In [10]:
def make_area_bucket(area):
    if area < 60: # 59타입
        return 0
    elif area < 85: # 84타입
        return 1
    else:
        return 2

all_data['bucket_area'] = all_data['전용면적'].apply(make_area_bucket)
all_data['bucket_area'].value_counts()

1    482741
0    445930
2    199423
Name: bucket_area, dtype: int64

- 한국의 아파트 유형은 59타입, 84타입의 유형이 많은 것으로 확인함
- 두 지점을 기준으로 전처리 후, 비율이 적절한지 확인함

### 5. 층수 전처리

In [ ]:
all_data['low_floor'] = all_data['층'].apply(lambda x: 0 if x <= 3 else 1)
all_data['low_floor'].value_counts()

- 3층을 기준으로 3층 이하일 경우, 저층으로 분류함

In [ ]:
# 결과 데이터프레임을 CSV 파일로 저장
output_filename = "../data/all_data.csv"
all_data.to_csv(output_filename, index=False)

print(f"Data saved to {output_filename}")